In [ ]:
# code modeled after https://github.com/madelinehayes/seabirdNET
# note: code is almost identical to Elephant_Seal_Annotations_Right_Format

In [ ]:
import os
import argparse
import numpy as np
import json
import csv
import random

In [ ]:
# change this to the dir for tiled images
path = '../Images/Harbor_Seals_Full_Data'
#path = '../Images/Harbor_Seals_Beach_Only'

image_list = [f for f in os.listdir(path) if f.endswith('.png')]

In [ ]:
# shuffle image list and split into training, testing, validation
random.shuffle(image_list)
total_count = len(image_list)

print(total_count)

# 80-10-10
test_index = int(total_count*.1)
val_index = int(total_count*.9*.1) + test_index

test_data = image_list[:test_index]
val_data = image_list[test_index:val_index]
train_data = image_list[val_index:]

print(len(test_data), len(val_data), len(train_data))

In [ ]:
# path to write the output file
via_path = "..\Images\Harbor_Seals_Full_Data\Harbor_Seal_Anno_csv.csv" 
image_annotations_train = []
image_annotations_test = []
image_annotations_val = []

with open(via_path, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader: 
        # output we want
        # path/to/image.jpg,x1,y1,x2,y2,class_name
        if '#' in line[0][0]:
            # bypassing comments in csv
            continue
        filename = line[1][2:-2]
        # strip brackets, split and get only the values we care about, then convert all the string to int 
        coords = list(map(int,list(map(float, line[4].strip('][').split(',')[1:]))))
        
        # make sure only bounding rectangles were used
        if len(coords) != 4:
            print('bad annotation')
            continue
            
        top_left_x, top_left_y, width, height = list(map(int,list(map(float, line[4].strip('][').split(',')[1:]))))

        # bad annotation (too small), so skip
        if width == 0 or height == 0:
            continue
        # move from top left and width/height to x and y values
        if top_left_x < 0:
            top_left_x = 1
        if top_left_y < 0:
            top_left_y = 1
        x1 = top_left_x
        x2 = top_left_x + width
        y1 = top_left_y
        y2 = top_left_y + height 
        
        # occassionally I had some issues with boxes being to 501 pixels, this just makes sense all boxes are on the image
        if x2 > 500:
            x2 = 500
        if y2 > 500:
            y2 = 500

        # only one class
        name = "seal"

        # create the csv row
        new_row = []
        if filename in train_data:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_train.append(new_row)
        elif filename in test_data:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_test.append(new_row)
        else:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_val.append(new_row)

In [ ]:
print(len(image_annotations_train), len(image_annotations_test), len(image_annotations_val))
print(len(image_annotations_train) + len(image_annotations_test) + len(image_annotations_val))

In [ ]:
# during training, these annotations must be in the same dir as the images being trained on
with open('..\Images\Harbor_Seals_Full_Data\harbor_test_annotations_full.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_test)
    
with open('..\Images\Harbor_Seals_Full_Data\harbor_val_annotations_full.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_val)
    
with open('..\Images\Harbor_Seals_Full_Data\harbor_train_annotations_full.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_train)

In [ ]:
detection_classes = [["seal", 0]]
with open('../Images/Harbor_Seals_Full_Data/harbor_classes.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(detection_classes)